In [180]:
!pip install rdflib >> /dev/null
!pip install pandas >> /dev/null
!pip install pydicom >> /dev/null
!pip install rdflib_jsonld >> /dev/null

El sistema no puede encontrar la ruta especificada.
El sistema no puede encontrar la ruta especificada.
El sistema no puede encontrar la ruta especificada.
El sistema no puede encontrar la ruta especificada.


In [181]:
import os

# Get the directory of the current script
base_dir = os.getcwd()
try:
    if folder:
        base_dir = os.path.join(base_dir, folder)
except:
    pass

In [182]:
from rdflib import * 
import uuid
from hashlib import sha256
import json

In [183]:
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
dqv = Namespace('http://www.w3.org/ns/dqv#')

In [184]:
class Federator:
    """
        Federator class is responsible for federating the data from different sources.
    """

    def __init__(self, ds, SDM):
        self.ds = ds
        self.sdm = SDM 

    def check_dp_existance(self):
        if self.sdm.value(predicate=RDF.type, subject=abox[self.ds]):
            return True 
        else:
            return False
        
    def generate_uri_id(self):
        return str(uuid.uuid4())
        
    def add_mappings(self, mappings):
        
        # create contract
        self.sdm.add((abox[f'dc_{self.ds}'],RDF.type, tbox.DataContract))
        self.sdm.add((abox[self.ds],tbox.hasDC, abox[f'dc_{self.ds}']))
        
        # add mappings
        for key, value in mappings.items(): # as key value pair dictionary
            
            # Generate mapping UUID
            mapping_uuid = self.generate_uri_id()
            self.sdm.add((abox[mapping_uuid], RDF.type, tbox.SchemaMapping))
            self.sdm.add((abox[f'dc_{self.ds}'],tbox.hasMapping, abox[mapping_uuid]))
            
            # Add Mapping
            self.sdm.add((abox[mapping_uuid], tbox.mfrom, abox[key]))
            self.sdm.add((abox[mapping_uuid], tbox.mto, abox[value]))
        
        return self.sdm
            
            
    def add_policies(self, policies):
        # Add agreed policies
        for policy in policies:
            self.sdm.add((abox[f'dc_{self.ds}'],tbox.hasPolicy, abox[policy]))
            
        return self.sdm
            
        
        

In [185]:
sdm = Graph().parse(os.path.join(base_dir, '../../FederatedComputationalGovernance/SemanticDataModel/sdm.ttl'), format='turtle')

In [186]:
dataset = input("Enter the dataset name: ")

In [187]:
federator = Federator(dataset, sdm)

In [188]:
print(federator.check_dp_existance())

True


Mappings and Policies are specified by the User

In [189]:
mappings = {
    'ID': 'Subject',
    'Age_at_scan_years': 'Age'
}

In [190]:
policies = ['p1', 'p2']

In [191]:
contract = federator.add_mappings(mappings)

In [192]:
contract = federator.add_policies(policies)

Save Contract that includes the specified Integration

In [193]:
sdm = Graph().parse(os.path.join(base_dir, '../../FederatedComputationalGovernance/SemanticDataModel/sdm.ttl'), format='turtle')
sdm += contract
sdm.serialize(destination=os.path.join(base_dir, '../../FederatedComputationalGovernance/SemanticDataModel/sdm.ttl'), format='turtle')

<Graph identifier=N4547f6209f8f458096169e6f90d0326d (<class 'rdflib.graph.Graph'>)>